In [1]:
!pip install gradio efficientnet-pytorch opencv-python-headless torch torchvision Pillow numpy

In [2]:
# Step 1: Install necessary packages (run this cell in Colab)
# !pip install gradio efficientnet-pytorch opencv-python-headless torch torchvision Pillow numpy

# Step 2: Import libraries
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import cv2 # OpenCV
import torchvision.models as models
import torch.nn as nn
import gradio as gr
import os # For checking file paths

# Optional: Mount Google Drive if your model is there (run this cell in Colab)
# from google.colab import drive
# drive.mount('/content/drive')

# --- Medical Information Store ---
# PLEASE NOTE: This information is generalized and for illustrative purposes ONLY.
# It is NOT a substitute for professional medical advice.
medical_conditions_info = {
    "polyps": {
        "display_name": "Polyps (General Information, often Colorectal)",
        "description": "Polyps are abnormal growths of tissue that most often look like small, flat bumps or tiny mushroom-like stalks. Most polyps are small and less than half an inch wide. Polyps in the colon are the most common, but it's also possible to develop polyps in places like the ear canal, cervix, stomach, nose, or throat. Most polyps are benign (noncancerous), but some can be precancerous or cancerous.",
        "symptoms": "Often, polyps don't cause symptoms. You might not know you have one until your doctor finds it during an examination. However, some people may experience: \n- Rectal bleeding (for colorectal polyps)\n- Change in bowel habits (constipation or diarrhea lasting more than a week)\n- Change in stool color (may show as red streaks or black, tarry stools)\n- Pain, nausea, or vomiting (if a large polyp partially obstructs the bowel)\n- Iron deficiency anemia (if polyps bleed slowly over time)",
        "treatment": "Treatment depends on the type, size, location, and whether polyps are cancerous or benign. \n- Observation: Small, benign polyps may not require immediate treatment but might be monitored.\n- Removal (Polypectomy): Most polyps can be removed during a colonoscopy or other endoscopic procedures. \n- Biopsy: Removed polyps are examined under a microscope to check for cancer.\n- Further surgery: If a polyp is too large to be removed endoscopically or if it's cancerous, more extensive surgery might be needed.",
        "prevention": "While not all polyps can be prevented, certain lifestyle choices may reduce the risk, particularly for colorectal polyps:\n- Eat a healthy diet rich in fruits, vegetables, and whole grains.\n- Limit red meat and processed meats.\n- Maintain a healthy weight.\n- Exercise regularly.\n- Don't smoke.\n- Limit alcohol consumption.\n- Get regular screenings (like colonoscopies) as recommended by your doctor, especially if you have risk factors like age or family history."
    },
    "dyed_lifted_polyps": { # Specific handling, but can point to general polyps info
        "display_name": "Dyed Lifted Polyps (Indicates a polyp prepared for removal/examination)",
        "description": "This finding indicates polyps that have been identified, dyed for better visibility, and potentially 'lifted' (e.g., by injecting saline underneath) in preparation for endoscopic removal (polypectomy) or biopsy. The key concern is the nature of the polyp itself.",
        "symptoms": "Refer to 'Polyps' symptoms. The finding itself is a result of a diagnostic/therapeutic procedure.",
        "treatment": "The treatment is typically the removal of these polyps, followed by pathological examination to determine if they are benign, precancerous, or cancerous. Further treatment depends on these findings. Refer to 'Polyps' treatment.",
        "prevention": "Refer to 'Polyps' prevention for reducing the risk of polyp formation."
    },
    "dyed_resection_margins": {
        "display_name": "Dyed Resection Margins (Status after surgical removal of tissue)",
        "description": "Resection margins refer to the edge of the tissue that was surgically removed (resected), often as part of cancer treatment (e.g., removing a tumor or polyps). The margins are 'dyed' by pathologists to help them orient and examine the tissue under a microscope. \n- 'Clear' or 'Negative' margins mean no cancer cells were found at the outer edge of the removed tissue, suggesting all the cancer was removed.\n- 'Positive' margins mean cancer cells are found at the edge, suggesting some cancer may have been left behind, potentially requiring further treatment like more surgery or radiation.",
        "symptoms": "This is not a disease itself but a finding from a pathology report after surgery. Symptoms would relate to the original condition for which surgery was performed.",
        "treatment": "If margins are positive, further treatment may be recommended, such as re-excision (more surgery to remove additional tissue), radiation therapy, or chemotherapy, depending on the type and stage of the cancer.",
        "prevention": "Prevention relates to the underlying condition that necessitated the resection. Achieving clear resection margins is a goal of cancer surgery."
    },
    "ulcerative_colitis": {
        "display_name": "Ulcerative Colitis",
        "description": "Ulcerative colitis (UC) is an inflammatory bowel disease (IBD) that causes long-lasting inflammation and ulcers (sores) in the innermost lining of your large intestine (colon) and rectum. It can be debilitating and can sometimes lead to life-threatening complications. There's no known cure, but treatment can greatly reduce signs and symptoms and bring about long-term remission.",
        "symptoms": "Symptoms can vary, depending on the severity of inflammation and where it occurs. They may include:\n- Diarrhea, often with blood or pus\n- Abdominal pain and cramping\n- Rectal pain\n- Rectal bleeding — passing small amount of blood with stool\n- Urgency to defecate\n- Inability to defecate despite urgency\n- Weight loss\n- Fatigue\n- Fever",
        "treatment": "Treatment aims to reduce inflammation, manage symptoms, and induce/maintain remission. It often involves:\n- Anti-inflammatory drugs (e.g., 5-aminosalicylates, corticosteroids)\n- Immune system suppressors (e.g., azathioprine, biologics like infliximab)\n- Other medications for symptoms like diarrhea or pain.\n- In severe cases, surgery to remove the colon and rectum (proctocolectomy) may be necessary.",
        "prevention": "There's no known way to prevent ulcerative colitis. However, managing the condition effectively with medical treatment can prevent complications. People with ulcerative colitis have an increased risk of colon cancer, so regular cancer screening (colonoscopies) is very important, typically starting several years after diagnosis."
    }
    # 'esophagus', 'normal_pylorus', 'normal_z_line' are considered normal/healthy
}


# --- Your Provided Inference Pipeline Code (Functions: apply_CLAHE, apply_Fourier, preprocess_image) ---
# Apply CLAHE
def apply_CLAHE(image_np): # Expects a NumPy array
    """Applies Contrast Limited Adaptive Histogram Equalization (CLAHE) to an image."""
    lab = cv2.cvtColor(image_np, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    cl = clahe.apply(l)
    lab_clahe = cv2.merge((cl, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)

# Apply Fourier Transform
def apply_Fourier(image_np): # Expects a NumPy array
    """Applies Fourier Transform and returns the magnitude spectrum as an image."""
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    f = np.fft.fft2(gray)
    fshift = np.fft.fftshift(f)
    magnitude_spectrum_raw = np.log(np.abs(fshift) + 1) # Add 1 to avoid log(0)

    min_val = np.min(magnitude_spectrum_raw)
    max_val = np.max(magnitude_spectrum_raw)
    if max_val == min_val:
        magnitude_spectrum_normalized = np.zeros_like(magnitude_spectrum_raw)
    else:
        magnitude_spectrum_normalized = (magnitude_spectrum_raw - min_val) / (max_val - min_val)

    magnitude_spectrum_uint8 = (magnitude_spectrum_normalized * 255).astype(np.uint8)
    return cv2.merge([magnitude_spectrum_uint8, magnitude_spectrum_uint8, magnitude_spectrum_uint8])

# Preprocessing function
def preprocess_image(pil_image): # Expects a PIL Image
    """Preprocesses the input PIL image for the model."""
    image_np = np.array(pil_image.convert('RGB'))

    clahe_img_np = apply_CLAHE(image_np)
    fourier_img_np = apply_Fourier(image_np)

    clahe_pil = Image.fromarray(clahe_img_np)
    fourier_pil = Image.fromarray(fourier_img_np)

    transform_pipeline = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])

    original_tensor = transform_pipeline(pil_image)
    clahe_tensor = transform_pipeline(clahe_pil)
    fourier_tensor = transform_pipeline(fourier_pil)

    processed_tensor = torch.cat((original_tensor, clahe_tensor, fourier_tensor), dim=0)

    if processed_tensor.ndim == 3:
        mean = torch.mean(processed_tensor, dim=[1, 2])
        std = torch.std(processed_tensor, dim=[1, 2]) + 1e-6
        mean = mean.unsqueeze(1).unsqueeze(2)
        std = std.unsqueeze(1).unsqueeze(2)
        normalized_tensor = (processed_tensor - mean) / std
    else:
        print(f"Warning: Unexpected tensor dimensions ({processed_tensor.ndim}), skipping normalization.")
        normalized_tensor = processed_tensor

    return normalized_tensor.unsqueeze(0)

# --- ModifiedEfficientNet Class ---
class ModifiedEfficientNet(nn.Module):
    def __init__(self, original_model, num_classes):
        super(ModifiedEfficientNet, self).__init__()
        self.channel_reduction = nn.Conv2d(9, 3, kernel_size=1, stride=1, padding=0)
        self.efficient_net = original_model
        in_features = self.efficient_net.classifier[1].in_features
        self.efficient_net.classifier = nn.Sequential(
            nn.Linear(in_features, 512),
            nn.ReLU(),
            nn.Dropout(p=0.5, inplace=True),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.channel_reduction(x)
        x = self.efficient_net(x)
        return x

# --- Model Loading and Setup ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_path = "C:/Users/vsneh/Videos/PROJECT/PROJ BESTT SO FAR/MODEL.pth" # UPDATE THIS PATH if necessary

if not os.path.exists(model_path):
    print(f"Error: Model file not found at {model_path}")
    # Consider how to handle this; Gradio app might not launch correctly.
    # For now, we'll let it proceed and error during model loading if the path is truly wrong.

try:
    base_model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)
except Exception as e:
    print(f"Error loading base EfficientNet-B0 model: {e}")
    raise

num_classes = 7
model = ModifiedEfficientNet(base_model, num_classes=num_classes)

try:
    state_dict = torch.load(model_path, map_location=device)
    new_state_dict = {}
    for k, v in state_dict.items():
        new_key = k.replace("module.", "")
        new_state_dict[new_key] = v
    model.load_state_dict(new_state_dict)
    print(f"Model state_dict loaded successfully from {model_path}")
except FileNotFoundError:
    print(f"ERROR: Model file not found at '{model_path}'. Please ensure the path is correct.")
    # In a real app, you might exit or disable the prediction functionality.
except Exception as e:
    print(f"Error loading model state_dict: {e}")
    raise

model = model.to(device)
model.eval()
print("Model loaded and set to evaluation mode.")

class_names = [
    'dyed_lifted_polyps',
    'dyed_resection_margins',
    'esophagus',
    'normal_pylorus',
    'normal_z_line',
    'polyps',
    'ulcerative_colitis'
]

# --- Prediction Function for Gradio (Updated) ---
def predict_gradio_with_info(pil_image_input):
    """
    Takes a PIL Image, predicts class, and returns confidences and medical info.
    """
    if pil_image_input is None:
        return {"Error": "No image provided."}, "Please upload an image."

    medical_info_markdown = "No specific information for this class, or it's a normal finding."

    try:
        image_rgb = pil_image_input.convert('RGB')
        input_tensor = preprocess_image(image_rgb).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            probabilities = torch.softmax(output, dim=1)[0]

        confidences = {class_names[i]: float(probabilities[i]) for i in range(len(class_names))}

        # Get the top predicted class
        top_class_idx = torch.argmax(probabilities).item()
        top_class_name = class_names[top_class_idx]

        # Retrieve medical information if available for the top class
        if top_class_name in medical_conditions_info:
            info = medical_conditions_info[top_class_name]
            medical_info_markdown = f"## {info['display_name']}\n\n"
            medical_info_markdown += f"**Description:**\n{info['description']}\n\n"
            medical_info_markdown += f"**Common Symptoms:**\n{info['symptoms']}\n\n"
            medical_info_markdown += f"**General Treatment Approaches:**\n{info['treatment']}\n\n"
            medical_info_markdown += f"**General Prevention Strategies:**\n{info['prevention']}\n\n"
            medical_info_markdown += "\n\n---\n*Disclaimer: This information is for general knowledge and illustrative purposes only. It is NOT a substitute for professional medical advice. Always consult with a qualified healthcare provider.*"
        elif top_class_name in ['esophagus', 'normal_pylorus', 'normal_z_line']:
             medical_info_markdown = f"**{top_class_name.replace('_', ' ').title()}**\n\nThis is generally considered a normal or healthy finding. No specific adverse condition information to display."

        return confidences, medical_info_markdown

    except Exception as e:
        print(f"Error during prediction: {e}")
        return {"Error": str(e)}, f"An error occurred during prediction: {str(e)}"

# --- Gradio Interface Definition (Updated with gr.Blocks) ---
if 'model' in globals() and model is not None:
    with gr.Blocks(theme=gr.themes.Soft()) as iface:
        gr.Markdown(
            """
            # Esophagus Cancer Detection and Classifier 🔬
            Upload an image to classify it. If a relevant medical condition is identified, general information about it will be displayed.
            **Disclaimer:** This tool is for informational purposes only and not a substitute for professional medical advice.
            """
        )
        with gr.Row():
            with gr.Column(scale=1):
                image_input = gr.Image(type="pil", label="Upload Image")
                submit_btn = gr.Button("Classify Image", variant="primary")
            with gr.Column(scale=2):
                predictions_output = gr.Label(num_top_classes=len(class_names), label="Predictions")
                medical_info_output = gr.Markdown(label="Condition Information")

        submit_btn.click(
            fn=predict_gradio_with_info,
            inputs=image_input,
            outputs=[predictions_output, medical_info_output]
        )

        gr.Examples(
            examples=[
                # Add paths to example images if you have them accessible by the script
                # e.g., "/content/drive/MyDrive/PROJECT/example_polyp.jpg",
            ],
            inputs=image_input,
            # outputs=[predictions_output, medical_info_output], # Optional: Define outputs for examples if needed
            # fn=predict_gradio_with_info, # Optional: function to run for examples
            cache_examples=False # Set to True if you want to precompute and cache example results
        )

    # --- Launch the Gradio App ---
    if __name__ == '__main__':
        print("Attempting to launch Gradio interface...")
        iface.launch(debug=True)
else:
    print("Model not loaded. Gradio interface cannot be launched.")


Using device: cpu
Model state_dict loaded successfully from C:/Users/vsneh/Videos/PROJECT/PROJ BESTT SO FAR/MODEL.pth
Model loaded and set to evaluation mode.
Attempting to launch Gradio interface...
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
